# Library

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

# Congfig

In [2]:
# konfigurasi
input_size = 784 # 28 x 28 (panjang x lebar)
hidden_size = 400 # jumlah neuron pada hidden layer
out_size = 10 # jumlah neuron pada output layer (0-9 : 10 digit)
batch_size = 100 # banyak sampel yang akan diproses dalam satu iterasi
epochs = 5
learning_rate = 0.001 # LR untuk optimizer

# Dataset

In [3]:
# unduh dataset MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [02:48<00:00, 59.0kB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 33.6kB/s]
100%|██████████| 1.65M/1.65M [00:23<00:00, 69.7kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.55MB/s]


In [7]:
# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
# lihat train loader
# for i, (images, labels) in enumerate(train_loader):
    # print(images.size())
    # images = images.view(-1, 28 * 28)  # ubah ukuran gambar menjadi vektor 1D
    # print(images.size())

    # output : (batch_size, 784)

# Model

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        '''
        Inisialisasi arstektur model 
        Args :
            input_size : ukuran fitur atau innput
            hidden_size : ukuran neuron pada hidden layer
            out_size : ukuran neuron untuk output (banyak kelas label/target)

            fc1, fc2, fc3 : tiga lapisan fully connected
            relu : aktivasi relu
        '''
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, out_size)
        self.relu = nn.ReLU()

        def forward(self, x):
            '''
            Proses forward
            '''
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
            out = self.relu(out)
            out = self.fc3(out)
            
            return out

In [9]:
net = Net(input_size, hidden_size, out_size)

CUDA = torch.cuda.is_available()

if CUDA:
    net = net.cuda()

In [11]:
# loss function
criterion = nn.CrossEntropyLoss() # comes along with sofmax

# optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Training loop

In [ ]:
correct_train = 0
total_train = 0

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # ubak ukuran gambar menjadi vektor 1D (batch, 1, 28, 28) -> (batch, 784)
        # Wrap ke variable
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        # pindahkan ke GPU
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()

        # kembalikan gradien ke nol
        optimizer.zero_grad()

        # forward
        outputs = net(images)

        # dapatkan indeks dari output dengan probabilitas tertinggi
        _, predicted = torch.max(outputs.data, 1)